In [1]:
!python3 -m pip install --force-reinstall sql_sidekick-0.1.7-py3-none-any.whl

Processing ./sql_sidekick-0.1.7-py3-none-any.whl
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached pandasql-0.7.3-py3-none-any.whl
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached loguru-0.7.2-py3-none-any.whl (62 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached psycopg2_binary-2.9.9-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached bitsandbytes-0.41.0-py3-none-any.whl (92.6 MB)
  Using cached SQLAlchemy_Utils-0.41.1-py3-none-any.whl (92 kB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached torch-2.0.1-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 KB 11.2 MB/s eta 0:00:00a 0:00:01
  Using cached sqlglot-12.4.0-py3-none-any.whl (259 kB)
  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
  Using cached Instruct

In [2]:
import gc
import json
import logging
import os
from pathlib import Path
from typing import List, Optional

import openai
import toml
import torch

In [3]:
from sidekick.prompter import (data_preview, db_setup, ask,
                               recommend_suggestions)
from sidekick.schema_generator import generate_schema
from sidekick.query import SQLGenerator
from sidekick.utils import (MODEL_CHOICE_MAP_DEFAULT,
                            MODEL_CHOICE_MAP_EVAL_MODE, TASK_CHOICE,
                            get_table_keys, save_query, setup_dir,
                            update_tables)

In [4]:
base_path = "./"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [5]:
# Setup the DB for a single table (WIP to extend to multiple tables)

HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
data_path = "./sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_demo"

r, table_info_path = generate_schema(data_path, f"{cache_path}/{table_name}_table_info.jsonl")

llm_response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path
            )

 Information supplied:
 querydb, localhost, sqlite, abc, 5432


2023-11-29 22:24:24.183 | DEBUG    | sidekick.db_config:__init__:42 - Creating SQLite DB: sqlite:////home/pramit/testing/q/apps/system/sql-sidekick/db/sqlite/querydb.db


Database already exists!
Table name: sleep_health_2


2023-11-29 22:24:24.189 | DEBUG    | sidekick.db_config:_extract_schema_info:100 - Schema path: /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_2_table_info.jsonl
2023-11-29 22:24:24.191 | DEBUG    | sidekick.db_config:create_table:145 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE


Checked table sleep_health_2 exists in the DB.


2023-11-29 22:24:24.197 | DEBUG    | sidekick.db_config:add_samples:179 - Adding sample values to table: /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv
2023-11-29 22:24:24.202 | DEBUG    | sidekick.db_config:add_samples:185 - Inserting chunk: 0
2023-11-29 22:24:24.492 | INFO     | sidekick.db_config:add_samples:190 - Data inserted into table: sleep_health_2
2023-11-29 22:24:24.495 | INFO     | sidekick.db_config:add_samples:195 - Number of rows inserted: 1122


Created a Database querydb. Inserted sample values from /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv into table sleep_health_2, please ask questions!


In [ ]:
def query(question: str, table_info_path: str, sample_qna_path: str):
    """Asks question and returns SQL."""
    base_path = "./"
    ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=None,
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder2",
        local_base_path=base_path
    )

In [13]:
# Zero-shot prompting
query("What is the average sleep duration for each gender?", table_info_path=table_info_path, sample_qna_path=None)

2023-11-29 22:28:56.382 | INFO     | sidekick.prompter:ask:421 - Table in use: ['sleep_health_2']
2023-11-29 22:28:56.384 | INFO     | sidekick.prompter:ask:453 - Setting context...
2023-11-29 22:28:56.385 | INFO     | sidekick.prompter:ask:454 - Question: What is the average sleep duration for each gender?
2023-11-29 22:28:56.387 | DEBUG    | sidekick.utils:is_resource_low:325 - Information on device: 0
2023-11-29 22:28:56.388 | INFO     | sidekick.utils:is_resource_low:328 - Total Memory: 23GB
2023-11-29 22:28:56.390 | INFO     | sidekick.utils:is_resource_low:329 - Free GPU memory: 12GB
2023-11-29 22:28:56.412 | INFO     | sidekick.utils:_check_file_info:435 - Using information info from path /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_2_table_info.jsonl
2023-11-29 22:28:56.422 | DEBUG    | sidekick.utils:semantic_search:125 - Input questions: # query: what is the average sleep duration for each gender?
2023-11-29 22:28:56.472 | DEBUG    | sidekick.utils

The query results are:
 [('Female', 7.229729729729707), ('Male', 7.03650793650794)]
Exiting...
